# Parametric simulation using accim predefined models

In [1]:
#todo import qgrid to manually change output dfs

In [2]:
import accim
from accim.parametric_and_optimisation.objectives import return_time_series
from besos import eppy_funcs as ef
from matplotlib import pyplot as plt
from accim.utils import print_available_outputs_mod
from accim.parametric_and_optimisation.main import OptimParamSimulation, get_rdd_file_as_df, get_mdd_file_as_df, parse_mtd_file
from os import listdir


C:\Users\usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's have a look at the files we currently have in the path:

In [3]:
original_files = [i for i in listdir()]
original_files

['.ipynb_checkpoints',
 'accim_predefined_model.ipynb',
 'Seville.epw',
 'Sydney.epw',
 'TestModel.idf',
 '__init__.py']

Firstly, the IDF must be read using besos's `get_building` function.

In [4]:
building = ef.get_building('TestModel.idf')

For this analysis, we want to use the HVAC system in all hours of the year, so that temperature is always comfortable. Therefore, we are going to set the occupancy to always on by means of the function `accim.utils.set_occupancy_to_always`, in which we input the IDF class instance we read in the previous cell.

In [5]:
accim.utils.set_occupancy_to_always(idf_object=building)

On 24/7 Schedule:Compact object was already in the model.
People Block1:Zone2 Number of People Schedule Name has been set to always occupied.
People Block1:Zone1 Number of People Schedule Name has been set to always occupied.


Now, let's start with the settings for the parametric analysis. First, let's instantiate the class `OptimParamSimulation`, and let's pass the IDF instance in the argument `building`. Argument `parameters_type` can take 3 different strings:
- "accim predefined model", in which models are those previously defined in accim (ComfStand=0 to ComfStand=22);
- "accim custom model", in which key parameters of the adaptive comfort model are defined in the relevant arguments;
- "apmv setpoints", in which setpoints are based on the aPMV (Adaptive Predicted Mean Vote) instead of the PMV index;

In this case, we're going to use the 'accim predefined model' type, in which the models we can use are those already defined in accim.

In [6]:
parametric = OptimParamSimulation(
    building=building,
    parameters_type='accim predefined model',
    #output_type='standard', #
    #output_keep_existing=False, #
    #output_freqs=['hourly'], #
    #ScriptType='vrf_mm', #
    #SupplyAirTempInputMethod='temperature difference', #
    #debugging=True, #
    #verbosemode=False #
)


--------------------------------------------------------
Adaptive-Comfort-Control-Implemented Model (ACCIM) v0.7.5
--------------------------------------------------------

This tool allows to apply adaptive setpoint temperatures. 
For further information, please read the documentation: 
https://accim.readthedocs.io/en/master/
For a visual understanding of the tool, please visit the following jupyter notebooks:
-    Using addAccis() to apply adaptive setpoint temperatures
https://accim.readthedocs.io/en/master/jupyter_notebooks/addAccis/using_addAccis.html
-    Using rename_epw_files() to rename the EPWs for proper data analysis after simulation
https://accim.readthedocs.io/en/master/jupyter_notebooks/rename_epw_files/using_rename_epw_files.html
-    Using runEp() to directly run simulations with EnergyPlus
https://accim.readthedocs.io/en/master/jupyter_notebooks/runEp/using_runEp.html
-    Using the class Table() for data analysis
https://accim.readthedocs.io/en/master/jupyter_notebo

An initial and generic version of the Adaptive-Comfort-Control-Implementation Script (ACCIS) has been added to the idf instance `building`. For instance, you can take a look at the parameter values accis currently has:

In [7]:
[i for i in building.idfobjects['energymanagementsystem:program'] if i.Name.lower() == 'setinputdata']

[
 ENERGYMANAGEMENTSYSTEM:PROGRAM,
     SetInputData,             !- Name
     set ComfStand = 1,        !- Program Line 1
     set CAT = 1,              !- Program Line 2
     set ComfMod = 2,          !- Program Line 3
     set HVACmode = 2,         !- Program Line 4
     set VentCtrl = 0,         !- Program Line 5
     set VSToffset = 0,        !- Program Line 6
     set MinOToffset = 7,      !- Program Line 7
     set MaxWindSpeed = 6,     !- Program Line 8
     set ACSTtol = -0.25,      !- Program Line 9
     set AHSTtol = 0.25,       !- Program Line 10
     set CoolSeasonStart = 121,    !- Program Line 11
     set CoolSeasonEnd = 274;    !- Program Line 12]

## Setting the outputs

### Outputs for the idf (i.e. the outputs for each simulation run)

First of all, we are going to set the outputs of the simulations that are going to be performed. This is an important step, especially if you are going to run hundreds or thousands of simulations.

Let's take a look at the Output:Variable objects we currently have in the idf. The method `get_output_var_df_from_idf()` returns a pandas DataFrame which contains the information of the existing Output:Variable objects in the idf:

In [8]:
df_output_variables_idf = parametric.get_output_var_df_from_idf()
df_output_variables_idf

,key_value,variable_name,reporting_frequency,schedule_name
0,*,Comfort Temperature,Hourly,
1,*,Adaptive Cooling Setpoint Temperature,Hourly,
2,*,Adaptive Heating Setpoint Temperature,Hourly,
3,*,Adaptive Cooling Setpoint Temperature_No Toler...,Hourly,
4,*,Adaptive Heating Setpoint Temperature_No Toler...,Hourly,
...,...,...,...,...
72,*,VRF Heat Pump Heating Electricity Energy,Hourly,
73,BLOCK1_ZONE2 VRF Indoor Unit DX Cooling Coil,Cooling Coil Total Cooling Rate,Hourly,
74,BLOCK1_ZONE2 VRF Indoor Unit DX Heating Coil,Heating Coil Heating Rate,Hourly,
75,BLOCK1_ZONE1 VRF Indoor Unit DX Cooling Coil,Cooling Coil Total Cooling Rate,Hourly,


now, let's see the Output:Meter objects:

In [9]:
df_output_meters_idf = parametric.get_output_meter_df_from_idf()
df_output_meters_idf.head()

,key_name,frequency


In this case, we can see there is no Output:Meter. However, there is a large number of Output:Variable objects which might result in heavy simulation outputs. So, let's get rid of some of them. We can drop the rows we want, and then input the modified DataFrame in the method `set_output_var_df_to_idf(outputs_df)`.

In [10]:
df_output_variables_idf = df_output_variables_idf[
    (
        df_output_variables_idf['variable_name'].str.contains('Setpoint Temperature_No Tolerance')
        |
        df_output_variables_idf['variable_name'].str.contains('Zone Operative Temperature')
        |
        df_output_variables_idf['variable_name'].str.contains('Zone Thermal Comfort ASHRAE 55 Adaptive Model Running Average Outdoor Air Temperature')
    )
]
df_output_variables_idf

,key_value,variable_name,reporting_frequency,schedule_name
3,*,Adaptive Cooling Setpoint Temperature_No Toler...,Hourly,
4,*,Adaptive Heating Setpoint Temperature_No Toler...,Hourly,
52,*,Zone Operative Temperature,Hourly,
54,*,Zone Thermal Comfort ASHRAE 55 Adaptive Model ...,Hourly,


Let's keep only the Output:Variable objects we have filtered using the `set_output_var_df_to_idf(outputs_df)`:

In [11]:
parametric.set_output_var_df_to_idf(outputs_df=df_output_variables_idf)

We have removed all rows except the adaptive heating and cooling setpoints, the operative temperature and the running mean outdoor temperature. Next optional step is adding Output:Meter objects. We can do that using the method `set_output_met_objects_to_idf(output_meters)`, where `output_meters` is a list of Output:Meter key names.

In [12]:
output_meters = [
    'Heating:Electricity',
    'Cooling:Electricity',
    'Electricity:HVAC',
]
parametric.set_output_met_objects_to_idf(output_meters=output_meters)

Let's see Output:Meter objects we currently have after adding these:

In [13]:
df_output_meters_idf = parametric.get_output_meter_df_from_idf()
df_output_meters_idf.head()

,key_name,frequency
0,Heating:Electricity,hourly
1,Cooling:Electricity,hourly
2,Electricity:HVAC,hourly


### Outputs to be read and shown in the parametric simulation or optimisation

To successfully run the parametric simulation or optimisation, it is advisable running a test simulation to know the outputs that each simulation will have. We can do that with the method `get_outputs_df_from_testsim()`, which returns a tuple containing 2 DataFrames containing respectively the Output:Meter and Output:Variable objects from the simulation. In this case, you won't find wildcards such as "*".

In [14]:
df_output_meters_testsim, df_output_variables_testsim = parametric.get_outputs_df_from_testsim()

In [15]:
df_output_meters_testsim

,key_name,frequency
0,Heating:Electricity,Hourly
1,Cooling:Electricity,Hourly
2,Electricity:HVAC,Hourly


In [16]:
df_output_variables_testsim

,key_value,variable_name,frequency
0,EMS,Adaptive Cooling Setpoint Temperature_No Toler...,Hourly
1,EMS,Adaptive Heating Setpoint Temperature_No Toler...,Hourly
2,BLOCK1:ZONE2,Zone Operative Temperature,Hourly
3,BLOCK1:ZONE1,Zone Operative Temperature,Hourly
4,PEOPLE BLOCK1:ZONE2,Zone Thermal Comfort ASHRAE 55 Adaptive Model ...,Hourly
5,PEOPLE BLOCK1:ZONE1,Zone Thermal Comfort ASHRAE 55 Adaptive Model ...,Hourly


We can get DataFrames from the .rdd and .mdd files generated from the test simulation using the functions `get_rdd_file_as_df()` and `get_mdd_file_as_df()`. 

In [17]:
df_rdd = get_rdd_file_as_df()
df_rdd

,object,key_value,variable_name,frequency,units
0,Output:Variable,*,Site Outdoor Air Drybulb Temperature,hourly,!- Zone Average [C]
1,Output:Variable,*,Site Outdoor Air Dewpoint Temperature,hourly,!- Zone Average [C]
2,Output:Variable,*,Site Outdoor Air Wetbulb Temperature,hourly,!- Zone Average [C]
3,Output:Variable,*,Site Outdoor Air Humidity Ratio,hourly,!- Zone Average [kgWater/kgDryAir]
4,Output:Variable,*,Site Outdoor Air Relative Humidity,hourly,!- Zone Average [%]
...,...,...,...,...,...
712,Output:Variable,*,Zone Ventilation When Unoccupied Time,hourly,!- HVAC Sum [hr]
713,Output:Variable,*,Facility Any Zone Ventilation Below Target Voz...,hourly,!- HVAC Sum [hr]
714,Output:Variable,*,Facility All Zones Ventilation At Target Voz Time,hourly,!- HVAC Sum [hr]
715,Output:Variable,*,Facility Any Zone Ventilation Above Target Voz...,hourly,!- HVAC Sum [hr]


In [18]:
df_mdd = get_mdd_file_as_df()
df_mdd

,object,meter_name,frequency,units
0,Output:Meter,Electricity:Facility,hourly,!- [J]
1,Output:Meter:Cumulative,Electricity:Facility,hourly,!- [J]
2,Output:Meter,Electricity:Building,hourly,!- [J]
3,Output:Meter:Cumulative,Electricity:Building,hourly,!- [J]
4,Output:Meter,Electricity:Zone:BLOCK1:ZONE2,hourly,!- [J]
...,...,...,...,...
157,Output:Meter:Cumulative,General:HeatRecovery:EnergyTransfer,hourly,!- [J]
158,Output:Meter,Carbon Equivalent:Facility,hourly,!- [kg]
159,Output:Meter:Cumulative,Carbon Equivalent:Facility,hourly,!- [kg]
160,Output:Meter,CarbonEquivalentEmissions:Carbon Equivalent,hourly,!- [kg]


Also, we can parse the .mtd files as a list using the function `parse_mtd_file()`.

In [19]:
mtd_list = parse_mtd_file()
mtd_list[0:2]

[{'meter_id': '14',
  'description': 'BLOCK1:ZONE2 GENERAL LIGHTING:Lights Electricity Energy [J]',
  'on_meters': ['Electricity:Facility [J]',
   'Electricity:Building [J]',
   'Electricity:Zone:BLOCK1:ZONE2 [J]',
   'Electricity:SpaceType:GENERAL [J]',
   'InteriorLights:Electricity [J]',
   'InteriorLights:Electricity:Zone:BLOCK1:ZONE2 [J]',
   'InteriorLights:Electricity:SpaceType:GENERAL [J]',
   'ELECTRIC EQUIPMENT#Block1:Zone2#GeneralLights:InteriorLights:Electricity [J]',
   'ELECTRIC EQUIPMENT#Block1:Zone2#GeneralLights:InteriorLights:Electricity:Zone:BLOCK1:ZONE2 [J]',
   'ELECTRIC EQUIPMENT#Block1:Zone2#GeneralLights:InteriorLights:Electricity:SpaceType:GENERAL [J]']},
 {'meter_id': '135',
  'description': 'BLOCK1:ZONE1 GENERAL LIGHTING:Lights Electricity Energy [J]',
  'on_meters': ['Electricity:Facility [J]',
   'Electricity:Building [J]',
   'Electricity:Zone:BLOCK1:ZONE1 [J]',
   'Electricity:SpaceType:GENERAL [J]',
   'InteriorLights:Electricity [J]',
   'InteriorLights

Therefore, we have 2 DataFrames, one for the Output:Meter and another for the Output:Variable objects. Next step is setting the outputs for the parametric simulation. To do so, we'll need to pass the DataFrames into the method `set_outputs_for_simulation(df_output_meter, df_output_variable)`. If you have some knowledge about the python package besos, you might think of these dataframes as if each row was a `MeterReader` or `VariableReader` instances respectively for the Output:Meter and Output:Variable dataframes, and the arguments in these were the specified in the columns. The `MeterReader` class takes the arguments `key_name`, `frequency`, `name` and `func`, while `VariableReader` class takes the arguments  `key_value`, `variable_name`, `frequency`, `name` and `func`.

In [20]:
[i for i in df_output_meters_testsim.columns]

['key_name', 'frequency']

In [21]:
[i for i in df_output_variables_testsim.columns]

['key_value', 'variable_name', 'frequency']

If you take a look at the columns of the dataframes above, you can see the names are the arguments in the `MeterReader` and `VariableReader` classes, and only `name` and `func` are missing. That means, you can add these columns to input the `name` and `func` arguments as desired. In case of the Output:Meter dataframe, we won't add the `name` and `func` columns, which means the name will be the `key_name` and hourly results will be aggregated using the pd.Series.sum() function. However, in case of the Output:Variable dataframe, we will specify these: we want the hourly values rather than the aggregation, therefore we will pass the name bound to the function `return_time_series`, and we will add '_time series' as a suffix to the `variable_name` column. We will also remove the outputs for BLOCK1:ZONE2, which are the rows 2 and 4.

In [22]:
df_output_variables_testsim['func'] = return_time_series
df_output_variables_testsim['name'] = df_output_variables_testsim['variable_name'] + '_time series'
df_output_variables_testsim = df_output_variables_testsim.drop(index=[2, 4])
df_output_variables_testsim

,key_value,variable_name,frequency,func,name
0,EMS,Adaptive Cooling Setpoint Temperature_No Toler...,Hourly,<function return_time_series at 0x000002427114...,Adaptive Cooling Setpoint Temperature_No Toler...
1,EMS,Adaptive Heating Setpoint Temperature_No Toler...,Hourly,<function return_time_series at 0x000002427114...,Adaptive Heating Setpoint Temperature_No Toler...
3,BLOCK1:ZONE1,Zone Operative Temperature,Hourly,<function return_time_series at 0x000002427114...,Zone Operative Temperature_time series
5,PEOPLE BLOCK1:ZONE1,Zone Thermal Comfort ASHRAE 55 Adaptive Model ...,Hourly,<function return_time_series at 0x000002427114...,Zone Thermal Comfort ASHRAE 55 Adaptive Model ...


Finally, let's set the outputs for parametric simulation and optimisation:

In [23]:
parametric.set_outputs_for_simulation(
    df_output_meter=df_output_meters_testsim,
    df_output_variable=df_output_variables_testsim,
)

If you want to inspect the `VariableReader` and `MeterReader` objects, you can see the internal variable `sim_outputs`:

In [27]:
parametric.sim_outputs

[MeterReader(name='Heating:Electricity', class_name='Output:Meter', frequency='Hourly', func=<function sum_values at 0x000002420171BA60>, key_name='Heating:Electricity'),
 MeterReader(name='Cooling:Electricity', class_name='Output:Meter', frequency='Hourly', func=<function sum_values at 0x000002420171BA60>, key_name='Cooling:Electricity'),
 MeterReader(name='Electricity:HVAC', class_name='Output:Meter', frequency='Hourly', func=<function sum_values at 0x000002420171BA60>, key_name='Electricity:HVAC'),
 VariableReader(name='Adaptive Cooling Setpoint Temperature_No Tolerance_time series', class_name='Output:Variable', frequency='Hourly', func=<function return_time_series at 0x0000024271140700>, key_value='EMS', variable_name='Adaptive Cooling Setpoint Temperature_No Tolerance'),
 VariableReader(name='Adaptive Heating Setpoint Temperature_No Tolerance_time series', class_name='Output:Variable', frequency='Hourly', func=<function return_time_series at 0x0000024271140700>, key_value='EMS', 

## Setting the parameters

At the top of the script, when you instantiated the class `OptimParamSimulation`, you already specified which type of parameters you were going to use. Now, the parameters we're about to set, must match the `parameters_type` argument. At this point, you may not know which parameters you can use, so you can call the method `get_available_parameters()`, which will return a list of available parameters:

In [29]:
available_parameters = parametric.get_available_parameters()
available_parameters

['ComfStand',
 'CAT',
 'CATcoolOffset',
 'CATheatOffset',
 'ComfMod',
 'SetpointAcc',
 'CoolSeasonStart',
 'CoolSeasonEnd',
 'HVACmode',
 'VentCtrl',
 'MaxTempDiffVOF',
 'MinTempDiffVOF',
 'MultiplierVOF',
 'VSToffset',
 'MinOToffset',
 'MaxWindSpeed',
 'ASTtol']

If you don't know what are these, please refer to the [documentation](https://accim.readthedocs.io/en/master/4_detailed%20use.html).

Using the 'accim predefined model' type, the values must be a list of options, since the values for some arguments are used to select some specific comfort model, and have a categorical use rather than numeric. Now, let's set the parameters using the method `set_parameters(accis_params_dict, additional_params)`. In this method, we set the parameters related to accim using the argument `accis_params_dict`, which takes a dictionary following the pattern {'parameter name': [1, 2, 3, etc]}. We can also add some other parameters not related to accim in the argument `additional_params`, which takes a list of parameters as if these were input straight to the besos EPProblem class. If additional arguments are added, the descriptors must be `CategoryParameter`.

In [30]:
accis_parameters = {
    'ComfStand': [1, 2],
    'CAT': [80, 90],
    'ComfMod': [3],
}
parametric.set_parameters(accis_params_dict=accis_parameters)

C:\Users\usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\besos\parameters.py:441: UserWarning: This parameter's descriptor is already named ComfStand.The name used as an input (ComfStand) will be discarded.
  warnings.warn(
C:\Users\usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\besos\parameters.py:441: UserWarning: This parameter's descriptor is already named CAT.The name used as an input (CAT) will be discarded.
  warnings.warn(
C:\Users\usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\besos\parameters.py:441: UserWarning: This parameter's descriptor is already named ComfMod.The name used as an input (ComfMod) will be discarded.
  warnings.warn(


If you want to inspect the `Parameter` objects, you can see the internal variable `parameters_list`:

In [31]:
parametric.parameters_list

[Parameter(selector=GenericSelector(set=<function modify_ComfStand at 0x000002421584A4C0>), value_descriptors=[CategoryParameter(name='ComfStand', options=[1, 2])]),
 Parameter(selector=GenericSelector(set=<function modify_CAT at 0x000002421584AB80>), value_descriptors=[CategoryParameter(name='CAT', options=[80, 90])]),
 Parameter(selector=GenericSelector(set=<function modify_ComfMod at 0x000002421584AD30>), value_descriptors=[CategoryParameter(name='ComfMod', options=[3])])]

## Running the parametric simulation

### Setting the problem

First, let's set the problem. To do so, use the `set_problem()` method. In case of the parametric simulation you don't need to input any argument. However, in case of the optimisation, you must input the arguments `minimize_outputs`, `constraints` and `constraint_bounds`, similarly as you would do in the besos `EPProblem` class.

In [32]:
parametric.set_problem()

Again, you can inspect the `EPProblem` class instance in the internal variable `problem`:

In [33]:
parametric.problem

EPProblem(inputs=[Parameter(selector=GenericSelector(set=<function modify_ComfStand at 0x000002421584A4C0>), value_descriptors=[CategoryParameter(name='ComfStand', options=[1, 2])]), Parameter(selector=GenericSelector(set=<function modify_CAT at 0x000002421584AB80>), value_descriptors=[CategoryParameter(name='CAT', options=[80, 90])]), Parameter(selector=GenericSelector(set=<function modify_ComfMod at 0x000002421584AD30>), value_descriptors=[CategoryParameter(name='ComfMod', options=[3])])], outputs=[MeterReader(name='Heating:Electricity', class_name='Output:Meter', frequency='Hourly', func=<function sum_values at 0x000002420171BA60>, key_name='Heating:Electricity'), MeterReader(name='Cooling:Electricity', class_name='Output:Meter', frequency='Hourly', func=<function sum_values at 0x000002420171BA60>, key_name='Cooling:Electricity'), MeterReader(name='Electricity:HVAC', class_name='Output:Meter', frequency='Hourly', func=<function sum_values at 0x000002420171BA60>, key_name='Electricit

### Sampling the simulation runs

The way to inform besos of the variations and permutations it must carry out in the parametric analysis is by means of a DataFrame, which must contain a column per `Parameter`, in which values are specified. There are multiple ways to do this DataFrame. For instance, we could make a dataframe from scratch:

In [38]:
import pandas as pd
param_dict = {'ComfStand': [2, 2], 'CAT': [80, 90], 'ComfMod': [3, 3]}
input_param_df = pd.DataFrame(data=param_dict)
input_param_df

,ComfStand,CAT,ComfMod
0,2,80,3
1,2,90,3


Also, we can use the sampling functions from besos (`full_factorial` and `lhs`), although these are not available using the accim predefined models, since these are based on ranges of values instead of options. In this case, we could use the `sampling_full_set()` method, which will combine all the values we entered when we set the parameters and drop the invalid combinations (e.g. you cannot use CAT=1, 2 or 3 if you are using ComfStand=2, since these CAT values are only available for ComfStand=1).

In [39]:
parametric.sampling_full_set()

Now, you can see the resulting input parameter dataframe in the internal variable `parameters_values_df`:

In [40]:
parametric.parameters_values_df

,ComfStand,CAT,ComfMod
2,2,80,3
3,2,90,3
